In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
from scipy.spatial.distance import cosine
from collections import defaultdict
from random import randint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


In this notebook, we predict new kinase-disease links using the RF classifier. 

#### Send your gmail address to vidarmehr@gmail.com to have access the files. 

## Load embeddings

In [ ]:
embedding_vectors = np.load("data/before2021/Jan2021/embedding_Skipgram_dim100.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

## Load pubmed words

In [ ]:
words = []
word_count = 0
with open("data/before2021/Jan2021/words.txt","r") as f:
    for line in f:
        word = line[2:-3]
        words.append(word)
        word_count += 1
print("number of words in pubmed abstracts:{}".format(word_count))       
        

## Create a dataframe of words and embeddings 

In [ ]:
embeddings = pd.DataFrame(data=embedding_vectors,index = words)
embeddings.head(n=10)

## Create a map meshid to disease

In [ ]:
disease_mesh = pd.read_csv("../input/neoplasms_labels.tsv",  sep= "\t", header = None)

In [ ]:
disease_mesh.head()

In [ ]:
meshid2disease_map = defaultdict()
for i in disease_mesh.index:
        mesh = disease_mesh.iloc[i][0]
        mesh_first_letter = mesh[0].lower()
        mesh_id = "mesh" + mesh_first_letter + mesh[1:]
        disease = disease_mesh.iloc[i][1]
        meshid2disease_map[mesh_id] = disease
        #print(mesh_id,disease)

## Create a map ncbigene to gene symbol

In [ ]:
kinase_gene_id = pd.read_csv("../input/prot_kinase.tsv",  sep= "\t", header = None)

In [ ]:
kinase_gene_id.head()

In [ ]:
ncbigene2symbol_map = defaultdict()
for i in kinase_gene_id.index:
    gene_symbol = kinase_gene_id.iloc[i][0]
    ncbigene = kinase_gene_id.iloc[i][2]
    ncbigene_id = "ncbigene" + str(ncbigene)
    ncbigene2symbol_map[ncbigene_id] = gene_symbol
    #print(ncbigene_id,gene_symbol)

## Positive training

In [ ]:
pos_train_data = pd.read_csv("../KCET_positive_2021.tsv",  sep= "\t")

In [ ]:
pos_train = pos_train_data[["mesh_id", "gene.id"]]

In [ ]:
pos_train.head()

## Calculate the difference between the kinases and mesh ids  

In [ ]:
diff_kinase_mesh_list_pos_train = []
diff_index_pos_train = []
for i in pos_train.index:
    ncbigene_id = pos_train.iloc[i][1]
    mesh_id = pos_train.iloc[i][0]
    if ncbigene_id in embeddings.index:
        ncbigene_id_embedding = embeddings.loc[ncbigene_id]
    else:
        print("The gene {} does not exist in Pubmed". format(ncbigene_id))
    if mesh_id in embeddings.index:
        mesh_id_embedding = embeddings.loc[mesh_id]
    else:
        print("The mesh id {} does not exist in Pubmed". format(mesh_id))   
    if ncbigene_id in embeddings.index and mesh_id in embeddings.index:     
        diff_kinase_mesh = np.subtract(ncbigene_id_embedding, mesh_id_embedding)
        diff_kinase_mesh_list_pos_train.append(diff_kinase_mesh)
        diff_index_pos_train.append(ncbigene_id + "," + mesh_id)

## Create a new dataframe, each row index is the ncbigene_id and mesh_id and the columns represent a diff vector between the vector of ncbigene_id and the vector of mesh_id

In [ ]:
diff_kinase_mesh_pos_train_data = pd.DataFrame(diff_kinase_mesh_list_pos_train, index = diff_index_pos_train) 
diff_kinase_mesh_pos_train_data.head()

In [ ]:
diff_kinase_mesh_pos_train_data.shape


#  Predictions

In [ ]:
prediction = pd.read_csv("../KCET_prediction_2021.tsv",  sep= "\t")

In [ ]:
prediction.head()

In [ ]:
prediction.shape

## Calculate the difference between the kinases and mesh id 

In [ ]:
diff_kinase_mesh_list_prediction = []
diff_index_prediction = []

for i in prediction.index:
    ncbigene_id = prediction.iloc[i][2]
    mesh_id = prediction.iloc[i][1]
    if ncbigene_id in embeddings.index:
        ncbigene_id_embedding = embeddings.loc[ncbigene_id]
    else:
        print("The gene {} does not exist in Pubmed". format(ncbigene_id))
    if mesh_id in embeddings.index:
        mesh_id_embedding = embeddings.loc[mesh_id]
    else:
        print("The mesh id {} does not exist in Pubmed". format(mesh_id)) 
    if ncbigene_id in embeddings.index and mesh_id in embeddings.index:    
        diff_kinase_mesh = np.subtract(ncbigene_id_embedding, mesh_id_embedding)
        diff_kinase_mesh_list_prediction.append(diff_kinase_mesh)
        diff_index_prediction.append(ncbigene_id + "," + mesh_id)


## Create a new dataframe, each row index is the ncbigene_id and mesh_id and the columns represent a diff vector between the vector of ncbigene_id and the vector of mesh_id

In [ ]:
diff_kinase_mesh_prediction_data = pd.DataFrame(diff_kinase_mesh_list_prediction, index = diff_index_prediction) 

In [ ]:
diff_kinase_mesh_prediction_data.head()


## Negative training 


In [ ]:
neg_train = pd.read_csv("../KCET_negative_2021.tsv",  sep= "\t")

In [ ]:
neg_train.head()

In [ ]:
neg_train.shape

## Calculate the difference between the kinases and mesh id 

In [ ]:
diff_kinase_mesh_list_neg_train = []
diff_index_neg_train = []
for i in neg_train.index:
    ncbigene_id = neg_train.iloc[i][2]
    mesh_id = neg_train.iloc[i][1]
    if ncbigene_id in embeddings.index:
        ncbigene_id_embedding = embeddings.loc[ncbigene_id]
    else:
        print("The gene {} does not exist in Pubmed". format(ncbigene_id))
    if mesh_id in embeddings.index:
        mesh_id_embedding = embeddings.loc[mesh_id]
    else:
        print("The disease {} does not exist in Pubmed". format(mesh_id)) 
    if ncbigene_id in embeddings.index and mesh_id in embeddings.index:    
        diff_kinase_mesh = np.subtract(ncbigene_id_embedding, mesh_id_embedding)
        diff_kinase_mesh_list_neg_train.append(diff_kinase_mesh)
        diff_index_neg_train.append(ncbigene_id + "," + mesh_id)


## Create a new dataframe, each row index is the ncbigene_id and mesh_id and the columns represent a diff vector between the vector of ncbigene_id and the vector of mesh_id

In [ ]:
diff_kinase_mesh_neg_train_data = pd.DataFrame(diff_kinase_mesh_list_neg_train, index = diff_index_neg_train) 

In [ ]:
diff_kinase_mesh_neg_train_data.head()

In [ ]:
diff_kinase_mesh_neg_train_data.shape

## Ceate trianing data by concatinating positive and negative training data

In [ ]:
train_data = [diff_kinase_mesh_pos_train_data,diff_kinase_mesh_neg_train_data]
X_train = pd.concat(train_data)

## Create labels for training data (label 1 for positive, 0 for negative data)

In [ ]:
label_1 = np.ones(diff_kinase_mesh_pos_train_data.shape[0])
label_0 = np.zeros(diff_kinase_mesh_neg_train_data.shape[0])
label_train = np.concatenate((label_1,label_0))
y_train = label_train

## Ceate test data (predictions) 

In [ ]:
X_test = diff_kinase_mesh_prediction_data

## Create labels for prediction data

In [ ]:
label_test = np.ones(diff_kinase_mesh_prediction_data.shape[0])
y_test = label_test

## Random Forest classifeir

In [ ]:
param_grid = {
                 'n_estimators': [10, 20, 50, 100, 150, 200],
                # 'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
                # 'max_depth' : np.linspace(1, 10, 5, endpoint=True),
             }

In [ ]:
clf=RandomForestClassifier()
random_clf = GridSearchCV(clf, param_grid, cv=10)
random_clf.fit(X_train,y_train)

In [ ]:
best_model = random_clf.best_estimator_

In [ ]:
y_pred=best_model.predict(X_test)
y_pred

In [ ]:
yproba = best_model.predict_proba(X_test)[::,1]
yproba

In [ ]:
X_test

In [ ]:
best_model.predict_proba(X_test)

In [ ]:
gene_symbol_list = []
cancer_list = []
for vec in X_test.index:
    fields = vec.split(",")
    ncbi_gene = fields[0]
    mesh_cancer = fields[1]
    #print(mesh_cancer)
    gene_symbol = ncbigene2symbol_map[ncbi_gene]
    gene_symbol_list.append(gene_symbol)
    cancer = meshid2disease_map[mesh_cancer]
    cancer_list.append(cancer)


In [ ]:
X_test.insert(0,"gene_symbol", gene_symbol_list, True)
X_test.insert(1,"cancer", cancer_list, True)
X_test.insert(2,"probability",yproba, True)

In [ ]:
X_test.head()

In [ ]:
sorted_X_test = X_test.sort_values(by=['probability'],ascending=False)
sorted_X_test.head()

In [ ]:
top_predictions = sorted_X_test[["gene_symbol","cancer","probability"]]

In [ ]:
top_predictions.head(n=20)

In [ ]:
top_predictions.to_csv("top_predictions_2021.tsv",index=False,sep="\t")

## Read dark kinases. The file was obtained from https://schurerlab.shinyapps.io/CKIApp/ 

In [ ]:
dark_kinase_list = pd.read_csv("dark_kinases.csv")["Gene"].to_list()
dark_kinase = set(dark_kinase_list)
print("There are {} dark kianses".format(len(dark_kinase)))

In [ ]:
dark_kinase

In [ ]:
dk_list = []
cancer_list = []
prob_list = []
index_list = []
#ratio_list = []
for i in range(top_predictions.shape[0]):
    if top_predictions.iloc[i]["gene_symbol"] in dark_kinase and top_predictions.iloc[i]["probability"] > 0:
        dk_list.append(top_predictions.iloc[i]["gene_symbol"])
        cancer_list.append(top_predictions.iloc[i]["cancer"])
        prob_list.append(top_predictions.iloc[i]["probability"])
        index_list.append(i)
        #ratio_list.append(i/top_predictions.shape[0])
        print(top_predictions.iloc[i]["gene_symbol"])
dark_kinase_cancer_prob = pd.DataFrame(list(zip(dk_list, cancer_list, prob_list, index_list)),columns= ["dark_kinase", 'cancer', 'probability', 'index(rank)'])
dark_kinase_cancer_prob.head()

In [ ]:
dark_kinase_cancer_prob.to_csv("dark_kinase_cancer_prob.tsv", sep ="\t", index= False)

## Histogram of probabilities 

In [ ]:
import matplotlib.pyplot as plt
from numpy import array
gn=array(top_predictions.loc[top_predictions["probability"] < 1]["probability"])
plt.hist(gn.astype('float'))
plt.show()

## Find dark kinases that have predicted LSI pairs

In [ ]:
def FindOverlapSliPubmedPredictions(sli_predictions_path,dark_kinase_cancer_predictions_path, common_kinases_output, score):
    gene1_sli = pd.read_csv(sli_predictions_path, sep = ",")['sources'].tolist()
    gene2_sli = pd.read_csv(sli_predictions_path, sep = ",")['destinations'].tolist()
    sli_score_list = pd.read_csv(sli_predictions_path, sep = ",")['score'].tolist()

    dark_kinase_cancer_predictions = pd.read_csv(dark_kinase_cancer_predictions_path, sep = "\t")
    dark_kinases_predicted = dark_kinase_cancer_predictions["dark_kinase"].tolist()
    cancer = dark_kinase_cancer_predictions["cancer"].tolist()
    prob = dark_kinase_cancer_predictions["probability"].tolist()
    print(len(gene1_sli))
    minimum_score = score
    common_kinase = []
    gene1 = []
    gene2 = []
    sli_score = []
    probability_score_RF = []
    cancers = []
    for i in range(len(dark_kinases_predicted)):
        for j in range(len(gene1_sli)):
            if gene1_sli[j] == dark_kinases_predicted[i] or gene2_sli[j] == dark_kinases_predicted[i]:
                if sli_score_list[j] > minimum_score:
                    common_kinase.append(dark_kinases_predicted[i])
                    gene1.append(gene1_sli[j])
                    gene2.append(gene2_sli[j])
                    sli_score.append(sli_score_list[j])
                    probability_score_RF.append(prob[i])
                    cancers.append(cancer[i])
                    #print(dark_kinases_predicted[j],gene1_sli[i],gene2_sli[i],dark_kinase_cancer_predictions.iloc[j]["cancer"],sli_score_list[i], dark_kinase_cancer_predictions.iloc[j]["probability"])
                    

    common_kinases = pd.DataFrame(list(zip(common_kinase,  probability_score_RF, cancers,gene1, gene2, sli_score )),
               columns =['dark_kinase', 'probability_RF_classifier', 'cancer','kinase_1_SLI', 'kinase_2_SLI', 'sli_score'])
    sorted_common_kinases = common_kinases.sort_values(by=['sli_score'],ascending=False)
    print(sorted_common_kinases.head(n=10))
    sorted_common_kinases.to_csv(common_kinases_output, sep = "\t", index = False)
    #print(common_genes_high_score.head(n=100))

In [ ]:
score = 0.0001
predicted_sli_path = "new_predicted_SLI_edges_from_depmap.csv"
dark_kinase_cancer_path = "dark_kinase_cancer_prob.tsv"
output_path = "common_kinases_sli_depmap_dark_kinase.tsv"
FindOverlapSliPubmedPredictions(predicted_sli_path,dark_kinase_cancer_path , 
                                output_path,score) 
                                

In [ ]:
score = 0.0001
predicted_sli_path = "new_predicted_SLI_edges_from_string_ppi.csv"
dark_kinase_cancer_path = "dark_kinase_cancer_prob.tsv"
output_path = "common_kinases_sli_string_ppi_dark_kinase.tsv"
FindOverlapSliPubmedPredictions(predicted_sli_path,dark_kinase_cancer_path , 
                                output_path,score) 